### Projektowanie Logiki Symulacji
#### Cele Symulacji
- Zaplanowanie zleceń produkcyjnych na podstawie zamówień sprzedaży.
- Obliczenie obciążenia gniazd roboczych.
- Wykrycie wąskich gardeł w produkcji.
- Symulacja zmian w ilości zamówień i ich wpływu na obciążenie gniazd.
#### Logika Symulacji: Kluczowe Kroki
1. Przygotowanie danych wejściowych:
- Połączenie danych marszruty z zamówieniami sprzedaży, aby określić operacje wymagane dla każdego zamówienia.
- Uwzględnienie dostępności gniazd i awarii.
2. Planowanie operacji:
- Kolejkowanie operacji w pparciu o marszrutę, ilość zamówioną i dostępne godziny gniazda roboczego.
3. Obliczenia obciążenia gniazd:
- Sumowanie zajętych godzin pracy (setup + run) dla każdego gniazda w danych dniach.
4. Analiza wąskich gardeł:
- Identyfikacja gniazd roboczych, które nie mogą obsłużyć operacji w wymaganym czasie.
5. Symulacje „Co jeśli”:
- Wzrost ilości zamówień sprzedaży (np. o 10%, 20%).
- Zmiana czasu przezbrojenia.
- Uwzględnienie awarii.

In [3]:
import pandas as pd
        
inventory = pd.read_csv('data/inventory.csv', sep = ',')


In [10]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990 entries, 0 to 1989
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Item ID           1990 non-null   object
 1   Item Description  1990 non-null   object
 2   Unit of Measure   1990 non-null   object
 3   Group             1864 non-null   object
 4   Item Type         1990 non-null   object
 5   Model             1864 non-null   object
dtypes: object(6)
memory usage: 93.4+ KB


In [11]:
import pandas as pd
        
routes = pd.read_csv('data/routes.csv', sep = ',')
routes

,Item ID,Operation Number,Operation Name,Workcenter ID,Setup Time (min),Run Time (min),Next Operation,Previous Operation
0,INV-STRC-FG-0001,1,Assembly 1,WC-014,18,338,2.0,NaN
1,INV-STRC-FG-0001,2,Welding 1,WC-001,20,180,3.0,1.0
2,INV-STRC-FG-0001,3,Assembly 2,WC-014,19,123,4.0,2.0
3,INV-STRC-FG-0001,4,Welding 2,WC-004,19,313,5.0,3.0
4,INV-STRC-FG-0001,5,Assembly 3,WC-015,23,244,6.0,4.0
...,...,...,...,...,...,...,...,...
8243,INV-MODL-SFG-1524,4,Welding 2,WC-003,29,338,NaN,3.0
8244,INV-MODL-SFG-1525,1,Assembly 1,WC-013,15,336,2.0,NaN
8245,INV-MODL-SFG-1525,2,Welding 1,WC-003,29,358,3.0,1.0
8246,INV-MODL-SFG-1525,3,Assembly 2,WC-013,18,171,4.0,2.0


In [13]:
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248 entries, 0 to 8247
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Item ID             8248 non-null   object 
 1   Operation Number    8248 non-null   int64  
 2   Operation Name      8248 non-null   object 
 3   Workcenter ID       8248 non-null   object 
 4   Setup Time (min)    8248 non-null   int64  
 5   Run Time (min)      8248 non-null   int64  
 6   Next Operation      6384 non-null   float64
 7   Previous Operation  6384 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 515.6+ KB


In [14]:
import pandas as pd
        
workcenters = pd.read_csv('data/workcenters.csv', sep = ',')
workcenters

,Workcenter ID,Workcenter Name,Department,Workstation Capacity,Employees per Station,Efficiency (%),Shifts,Availability (Hours)
0,WC-001,WEL-01,Welding,4,1,90,2,16
1,WC-002,WEL-02,Welding,4,2,77,2,16
2,WC-003,WEL-03,Welding,2,1,82,2,16
3,WC-004,WEL-04,Welding,4,2,83,2,16
4,WC-005,WEL-05,Welding,4,1,77,2,16
5,WC-006,WEL-06,Welding,5,1,77,2,16
6,WC-007,MAC-07,Machining,2,1,79,2,16
7,WC-008,MAC-08,Machining,2,1,81,2,16
8,WC-009,MAC-09,Machining,3,1,78,2,16
9,WC-010,MAC-10,Machining,4,1,77,2,16


In [19]:
import pandas as pd
        
inventory = pd.read_csv('data/inventory.csv', sep = ',')
bom = pd.read_csv('data/bom.csv', sep = ',')
routes = pd.read_csv('data/routes.csv', sep = ',')
workcenters = pd.read_csv('data/workcenters.csv', sep = ',')
workcenter_availabiity = pd.read_csv('data/workcenter_availabiity.csv', sep = ',')

In [22]:
import openpyxl

inventory.to_excel('data/inventory.xlsx')
bom.to_excel('data/bom.xlsx')
routes.to_excel('data/routes.xlsx')
workcenters.to_excel('data/workcenters.xlsx')
workcenter_availabiity.to_excel('data/workcenter_availabiity.xlsx')

In [23]:
import pandas as pd
        
sales_orders = pd.read_csv('data/sales_orders.csv', sep = ',')
sales_orders

,Order ID,Customer ID,Item ID,Quantity,Delivery Date
0,SO-0001,CUST-006,INV-STRC-FG-0058,2,2025-01-28
1,SO-0002,CUST-012,INV-PLTE-FG-0260,1,2025-10-21
2,SO-0003,CUST-010,INV-PLTE-FG-0258,4,2025-12-19
3,SO-0004,CUST-010,INV-PLTE-FG-0200,4,2025-12-22
4,SO-0005,CUST-004,INV-BEAM-FG-0117,2,2025-09-18
...,...,...,...,...,...
995,SO-0996,CUST-002,INV-STRC-FG-0033,1,2025-03-20
996,SO-0997,CUST-006,INV-BEAM-FG-0112,4,2025-10-14
997,SO-0998,CUST-011,INV-MODL-FG-0268,2,2025-03-05
998,SO-0999,CUST-012,INV-MODL-FG-0311,1,2025-03-07


In [26]:
sales_orders.describe(include='all')

,Order ID,Customer ID,Item ID,Quantity,Delivery Date
count,1000,1000,1000,1000.000000,1000
unique,1000,12,317,NaN,332
top,SO-0001,CUST-011,INV-BEAM-FG-0118,NaN,2025-12-16
freq,1,101,9,NaN,9
mean,NaN,NaN,NaN,1.307000,NaN
std,NaN,NaN,NaN,0.707992,NaN
min,NaN,NaN,NaN,1.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN
50%,NaN,NaN,NaN,1.000000,NaN
75%,NaN,NaN,NaN,1.000000,NaN
